In [1]:
import pandas as pd
import numpy as np
import matplotlib as plot
import seaborn as sns
import requests
import json

In [6]:
#reading the dataset
df= pd.read_csv("honduras_triage_anon_data(Local)copy.csv")

/var/folders/34/v3dd8zl14rl974rs2qmpy8zh0000gn/T/ipykernel_21370/1587428441.py:2: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("honduras_triage_anon_data(Local)copy.csv")


In [7]:
df.shape

(106906, 62)

In [8]:
# Reading only the first 50 rows and selecting specific columns
df_to_be_translated = df[['ADMISSION_NOTES', 'TRIAGE_NOTES', 'BACKGROUND_TRIAGE', 'OTHER_ALLERGIES']]

# Saving the first 50 rows with selected columns to a demo file
demo_file_path = 'To_be_translated.csv'  # Changed the file extension to .xlsx
df_to_be_translated.to_csv(demo_file_path, index=False)

print("File created successfully!")



File created successfully!


In [11]:
api_key = 'Your Key'

Testing if the key is valid:

In [12]:
 import requests
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

def test_api_key():
    try:
        response = requests.get('https://api.openai.com/v1/engines', headers=headers)
        if response.status_code == 200:
            print("API key is valid.")
            print("Available engines:", response.json())
        else:
            print(f"Failed to validate API key. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error occurred while testing API key: {e}")

# Test the API key
test_api_key()

API key is valid.
Available engines: {'object': 'list', 'data': [{'object': 'engine', 'id': 'whisper-1', 'ready': True, 'owner': 'openai-internal', 'permissions': None, 'created': None}, {'object': 'engine', 'id': 'gpt-4-turbo-2024-04-09', 'ready': True, 'owner': 'system', 'permissions': None, 'created': None}, {'object': 'engine', 'id': 'davinci-002', 'ready': True, 'owner': 'system', 'permissions': None, 'created': None}, {'object': 'engine', 'id': 'gpt-3.5-turbo', 'ready': True, 'owner': 'openai', 'permissions': None, 'created': None}, {'object': 'engine', 'id': 'gpt-4-turbo', 'ready': True, 'owner': 'system', 'permissions': None, 'created': None}, {'object': 'engine', 'id': 'dall-e-2', 'ready': True, 'owner': 'system', 'permissions': None, 'created': None}, {'object': 'engine', 'id': 'gpt-3.5-turbo-16k', 'ready': True, 'owner': 'openai-internal', 'permissions': None, 'created': None}, {'object': 'engine', 'id': 'tts-1-hd-1106', 'ready': True, 'owner': 'system', 'permissions': None,

Doing a simple query to get an output:

In [41]:
import requests

def get_chatgpt_response(prompt):
    url = "https://api.openai.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        'Authorization': f'Bearer {api_key}' # Replace with your actual API key
    }
    data = {
        "model": "gpt-3.5-turbo-instruct",  # You can choose a different model here if needed---- "davinci-002"
        "prompt": prompt,
        "max_tokens": 60  # Adjust the number of tokens for response length
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["text"]
    else:
        print("Failed to get response. Status code:", response.status_code)
        return None

# Example usage
prompt = "Hello GPT!"
response = get_chatgpt_response(prompt)
if response:
    print("GPT-3 says:", response)


GPT-3 says:  How are you today

I am an AI and do not have emotions like humans, but thank you for asking! I am functioning as expected and ready to assist you with any questions or tasks you may have. How can I help you?


Testing all the engines available to us:

In [16]:
# import requests

# # Dictionary to map engine names to their respective Engine ID and Owner (list of engines we have access to)
# engines = {
#     "whisper-1": "Engine ID: whisper-1 - Owner: openai-internal",
#     "davinci-002": "Engine ID: davinci-002 - Owner: system",
#     "gpt-3.5-turbo": "Engine ID: gpt-3.5-turbo - Owner: openai",
#     "dall-e-2": "Engine ID: dall-e-2 - Owner: system",
#     "gpt-3.5-turbo-16k": "Engine ID: gpt-3.5-turbo-16k - Owner: openai-internal",
#     "gpt-3.5-turbo-16k-0613": "Engine ID: gpt-3.5-turbo-16k-0613 - Owner: openai",
#     "tts-1-hd-1106": "Engine ID: tts-1-hd-1106 - Owner: system",
#     "tts-1-hd": "Engine ID: tts-1-hd - Owner: system",
#     "text-embedding-3-large": "Engine ID: text-embedding-3-large - Owner: system",
#     "gpt-4-1106-vision-preview": "Engine ID: gpt-4-1106-vision-preview - Owner: system",
#     "gpt-3.5-turbo-instruct-0914": "Engine ID: gpt-3.5-turbo-instruct-0914 - Owner: system",
#     "gpt-4-0125-preview": "Engine ID: gpt-4-0125-preview - Owner: system",
#     "gpt-4-turbo-preview": "Engine ID: gpt-4-turbo-preview - Owner: system",
#     "gpt-3.5-turbo-instruct": "Engine ID: gpt-3.5-turbo-instruct - Owner: system",
#     "gpt-3.5-turbo-0301": "Engine ID: gpt-3.5-turbo-0301 - Owner: openai",
#     "gpt-3.5-turbo-0613": "Engine ID: gpt-3.5-turbo-0613 - Owner: openai",
#     "tts-1": "Engine ID: tts-1 - Owner: openai-internal",
#     "dall-e-3": "Engine ID: dall-e-3 - Owner: system",
#     "gpt-3.5-turbo-1106": "Engine ID: gpt-3.5-turbo-1106 - Owner: system",
#     "gpt-4-1106-preview": "Engine ID: gpt-4-1106-preview - Owner: system",
#     "babbage-002": "Engine ID: babbage-002 - Owner: system",
#     "tts-1-1106": "Engine ID: tts-1-1106 - Owner: system",
#     "gpt-4-vision-preview": "Engine ID: gpt-4-vision-preview - Owner: system",
#     "text-embedding-3-small": "Engine ID: text-embedding-3-small - Owner: system",
#     "gpt-4": "Engine ID: gpt-4 - Owner: openai",
#     "text-embedding-ada-002": "Engine ID: text-embedding-ada-002 - Owner: openai-internal",
#     "gpt-3.5-turbo-0125": "Engine ID: gpt-3.5-turbo-0125 - Owner: system",
#     "gpt-4-0613": "Engine ID: gpt-4-0613 - Owner: openai"
# }

# def get_chatgpt_response(prompt, engine_id):
#     url = "https://api.openai.com/v1/completions"
#     headers = {
#         "Content-Type": "application/json",
#         'Authorization': f'Bearer {api_key}' # Replace with your actual API key
#     }
#     data = {
#         "model": engine_id,
#         "prompt": prompt,  # Change prompt_name to prompt
#         "max_tokens": 20
#     }
    
#     response = requests.post(url, headers=headers, json=data)
#     if response.status_code == 200:
#         return response.json()["choices"][0]["text"]
#     else:
#         print(f"{engines[engine_id]} - Failed to get response. Status code:", response.status_code)
#         return None

# # Example usage
# prompt = "Hello GPT!"
# for engine_id in engines:
#     response = get_chatgpt_response(prompt, engine_id)
#     if response:
#         print(f"{engines[engine_id]} - engine says:", response)


Summarization:

| Engine ID            | Owner             | Response                                                                                                                                                                 |
|----------------------|-------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| whisper-1            | openai-internal   | Failed to get response. Status code: 404                                                                                                                                |
| davinci-002          | system            | How are you? Hope all is well on your side of the world. "No more gas, no                                                                                               |
| gpt-3.5-turbo        | openai            | Failed to get response. Status code: 404                                                                                                                                |
| dall-e-2             | system            | Failed to get response. Status code: 403                                                                                                                                |
| gpt-3.5-turbo-16k    | openai-internal   | Failed to get response. Status code: 404                                                                                                                                |
| gpt-3.5-turbo-16k-0613| openai            | Failed to get response. Status code: 404                                                                                                                                |
| tts-1-hd-1106        | system            | Failed to get response. Status code: 404                                                                                                                                |
| tts-1-hd             | system            | Failed to get response. Status code: 404                                                                                                                                |
| text-embedding-3-large| system           | Failed to get response. Status code: 403                                                                                                                                |
| gpt-4-1106-vision-preview| system        | Failed to get response. Status code: 400                                                                                                                                |
| gpt-3.5-turbo-instruct-0914| system       | Hello there! My name is GPT (Generative Pre-trained Transformer), I am an AI                                                                                             |
| gpt-4-0125-preview   | system            | Failed to get response. Status code: 404                                                                                                                                |
| gpt-4-turbo-preview  | system            | Failed to get response. Status code: 404                                                                                                                                |
| gpt-3.5-turbo-instruct| system           | Hello there! I am just a language model AI, so I don't have personal emotions or                                                                                         |
| gpt-3.5-turbo-0301   | openai            | Failed to get response. Status code: 404                                                                                                                                |
| gpt-3.5-turbo-0613   | openai            | Failed to get response. Status code: 404                                                                                                                                |
| tts-1                | openai-internal   | Failed to get response. Status code: 404                                                                                                                                |
| dall-e-3             | system            | Failed to get response. Status code: 403                                                                                                                                |
| gpt-3.5-turbo-1106   | system            | Failed to get response. Status code: 404                                                                                                                                |
| gpt-4-1106-preview   | system            | I’m thinking of using one of your models to train an RL agent to play Dance Dance Revolution against                                                                     |
| babbage-002          | system            | VIP will not let you broadcast. Like that so you can conduct poll and ask questions ps I dont                                                                            |
| tts-1-1106           | system            | Failed to get response. Status code: 404                                                                                                                                |
| gpt-4-vision-preview | system            | Failed to get response. Status code: 400                                                                                                                                |
| text-embedding-3-small| system           | Failed to get response. Status code: 403                                                                                                                                |
| gpt-4                | openai            | Failed to get response. Status code: 404                                                                                                                                |
| text-embedding-ada-002| openai-internal  | GPTLG,GptlG, TlGptTmmGmGm                                                                                                                                                |
| gpt-3.5-turbo-0125   | system            | Failed to get response. Status code: 404                                                                                                                                |
| gpt-4-0613           | openai            | Failed to get response. Status code: 404                                                                                                                                 |

The ones that somewhat work effectively are:
- davinci-002
- gpt-3.5-turbo-instruct-0914
- gpt-3.5-turbo-instruct

In [9]:
## testing with just one row

import pandas as pd

# Read the first row from the CSV file
df = pd.read_csv('To_be_translated.csv', nrows=1)


In [ ]:
# lets try getting a response from one row and one colum (one cell) and it works

In [12]:
import requests
from googletrans import Translator
import pandas as pd

# Function to translate text from Spanish to English using Google Translate API
def translate_to_english(text):
    translator = Translator()
    translated = translator.translate(text, src='es', dest='en')
    return translated.text

def get_chatgpt_response(prompt):
    url = "https://api.openai.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        'Authorization': f'Bearer {api_key}' # Replace with your actual API key
    }
    data = {
        "model": "gpt-3.5-turbo-instruct",  # You can choose a different model here if needed---- "davinci-002"
        "prompt": prompt,
        "max_tokens": 60  # Adjust the number of tokens for response length
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["text"]
    else:
        print("Failed to get response. Status code:", response.status_code)
        return None

# Example usage with a DataFrame row
def process_df_row(row):
    # Translate the row from Spanish to English
    translated_text = translate_to_english(row['ADMISSION_NOTES'])  # Change 'spanish_column' to your actual column name
    
    # Get GPT-3 response based on translated text
    response = get_chatgpt_response(translated_text)
    
    return response

# Example usage with a DataFrame

# Process each row of the DataFrame
df['response'] = df.apply(process_df_row, axis=1)


In [ ]:
# lets try one row, 4 columns 

In [44]:
api_key = 'Your Key'

In [13]:
import pandas as pd
from googletrans import Translator

# Function to translate text from Spanish to English using Google Translate API
def translate_to_english(text):
    translator = Translator()
    translated = translator.translate(text, src='es', dest='en')
    return translated.text

def get_chatgpt_response(prompt):
    
    url = "https://api.openai.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        'Authorization': f'Bearer {api_key}' 
    }
    data = {
        "model": "gpt-3.5-turbo-instruct",
        "prompt": prompt,
        "max_tokens": 60
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["text"]
    else:
        print("Failed to get response. Status code:", response.status_code)
        return None

# Function to process each row of the DataFrame
def process_df_row(row):
    translated_columns = {}
    for col in ['ADMISSION_NOTES', 'TRIAGE_NOTES', 'BACKGROUND_TRIAGE', 'OTHER_ALLERGIES']:
        translated_text = translate_to_english(row[col])
        generated_text = get_chatgpt_response(translated_text)
        translated_columns[col+'_translated'] = generated_text
    return translated_columns


# Apply translation and GPT-3 response generation on each row
translated_df = df.apply(process_df_row, axis=1, result_type='expand')

# Concatenate the original DataFrame with the translated DataFrame
final_df = pd.concat([df, translated_df], axis=1)

# # Print the final DataFrame
# print(final_df)


In [21]:
final_df.to_csv('demo_translated.csv')

Observation: Have to adjust for Nan and the translation are very bizzare

In [ ]:
# lets remove google translate API

In [14]:
## testing with just one row

import pandas as pd

# Read the first row from the CSV file
df = pd.read_csv('To_be_translated.csv', nrows=1)

# # Print the dataframe
# print(df)


In [24]:
import pandas as pd
import requests

# Function to get response from ChatGPT using a given prompt
def get_chatgpt_response(prompt):
    url = "https://api.openai.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        'Authorization': f'Bearer {api_key}' 
    }
    data = {
        "model": "davinci-002",  # Adjust the model as needed
        "prompt": prompt,
        "max_tokens": 60
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["text"]
    else:
        print("Failed to get response. Status code:", response.status_code)
        return None

# Function to process each row of the DataFrame
def process_df_row(row, prompt):
    translated_columns = {}
    for col in ['ADMISSION_NOTES', 'TRIAGE_NOTES', 'BACKGROUND_TRIAGE', 'OTHER_ALLERGIES']:
        generated_text = get_chatgpt_response(prompt + ' ' + row[col])
        translated_columns[col+'_translated'] = generated_text
    return translated_columns

# Function to interactively get the prompt from the user
def get_user_prompt():
    return input("Enter your prompt: ")

# Get user prompt
user_prompt = get_user_prompt()

# Apply translation and GPT-3 response generation on each row with the user's prompt
translated_df = df.apply(process_df_row, axis=1, result_type='expand', prompt=user_prompt)

# Concatenate the original DataFrame with the translated DataFrame
final_df = pd.concat([df, translated_df], axis=1)

# Print the final DataFrame
print(final_df)


Enter your prompt: This is a spanish medical dataset, I want it translated into english. Every word is in spanish and needs translation. Do not add any additional information, just translate what is written.


TypeError: can only concatenate str (not "float") to str

In [25]:
##Need to adjust for the Nan

In [15]:
## testing with just one row

import pandas as pd

# Read the first row from the CSV file
df = pd.read_csv('To_be_translated.csv', nrows=1)


In [16]:
import pandas as pd
import requests
import numpy as np

# Function to get response from ChatGPT using a given prompt
def get_chatgpt_response(prompt):
    url = "https://api.openai.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        'Authorization': f'Bearer {api_key}' 
    }
    data = {
        "model": "gpt-3.5-turbo-instruct",  # Adjust the model as needed
        "prompt": prompt,
        "max_tokens": 60
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["text"]
    else:
        print("Failed to get response. Status code:", response.status_code)
        return None

# Function to process each row of the DataFrame
def process_df_row(row, prompt):
    translated_columns = {}
    for col in ['ADMISSION_NOTES', 'TRIAGE_NOTES', 'BACKGROUND_TRIAGE', 'OTHER_ALLERGIES']:
        if not pd.isna(row[col]):  # Check if the value is not NaN
            generated_text = get_chatgpt_response(prompt + ' ' + row[col])
            translated_columns[col+'_translated'] = generated_text
    return translated_columns

# Function to interactively get the prompt from the user
def get_user_prompt():
    return input("Enter your prompt: ")

# Get user prompt
user_prompt = get_user_prompt()

# Apply translation and GPT-3 response generation on each row with the user's prompt
translated_df = df.apply(process_df_row, axis=1, result_type='expand', prompt=user_prompt)

# Concatenate the original DataFrame with the translated DataFrame
final_df = pd.concat([df, translated_df], axis=1)

# # Print the final DataFrame
# print(final_df)


Enter your prompt: Enter your prompt: This is a spanish medical dataset, I want it translated into english. Every word is in spanish and needs translation. Do not add any additional information, just translate what is written.


In [17]:
# final_df

In [ ]:
## perfect but it completly ignores the column having a Nan

In [18]:
import pandas as pd
import requests
import numpy as np

# Function to get response from ChatGPT using a given prompt
def get_chatgpt_response(prompt):
    url = "https://api.openai.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        'Authorization': f'Bearer {api_key}' 
    }
    data = {
        "model": "gpt-3.5-turbo-instruct",  # Adjust the model as needed
        "prompt": prompt,
        "max_tokens": 60
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["text"]
    else:
        print("Failed to get response. Status code:", response.status_code)
        return None

# Function to process each row of the DataFrame
def process_df_row(row, prompt):
    translated_columns = {}
    for col in ['ADMISSION_NOTES', 'TRIAGE_NOTES', 'BACKGROUND_TRIAGE', 'OTHER_ALLERGIES']:
        if pd.notna(row[col]):  # Check if the value is not NaN
            generated_text = get_chatgpt_response(prompt + ' ' + row[col])
            translated_columns[col+'_translated'] = generated_text
        else:
            translated_columns[col+'_translated'] = np.nan
    return translated_columns

# Function to interactively get the prompt from the user
def get_user_prompt():
    return input("Enter your prompt: ")

# Get user prompt
user_prompt = get_user_prompt()

# Apply translation and GPT-3 response generation on each row with the user's prompt
translated_df = df.apply(process_df_row, axis=1, result_type='expand', prompt=user_prompt)

# Concatenate the original DataFrame with the translated DataFrame
final_df = pd.concat([df, translated_df], axis=1)

# # Print the final DataFrame
# print(final_df)


Enter your prompt: This is a spanish medical dataset, I want it translated into english. Every word is in spanish and needs translation. Do not add any additional information, just translate what is written.


In [ ]:
# lets try first 10 rows

In [19]:
## testing with just one row

import pandas as pd

# Read the first row from the CSV file
df = pd.read_csv('To_be_translated.csv', nrows=10)


In [20]:
## Using the same code as above

import pandas as pd
import requests
import numpy as np

# Function to get response from ChatGPT using a given prompt
def get_chatgpt_response(prompt):
    url = "https://api.openai.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        'Authorization': f'Bearer {api_key}' 
    }
    data = {
        "model": "gpt-3.5-turbo-instruct",  # Adjust the model as needed
        "prompt": prompt,
        "max_tokens": 60
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["text"]
    else:
        print("Failed to get response. Status code:", response.status_code)
        return None

# Function to process each row of the DataFrame
def process_df_row(row, prompt):
    translated_columns = {}
    for col in ['ADMISSION_NOTES', 'TRIAGE_NOTES', 'BACKGROUND_TRIAGE', 'OTHER_ALLERGIES']:
        if pd.notna(row[col]):  # Check if the value is not NaN
            generated_text = get_chatgpt_response(prompt + ' ' + row[col])
            translated_columns[col+'_translated'] = generated_text
        else:
            translated_columns[col+'_translated'] = np.nan
    return translated_columns

# Function to interactively get the prompt from the user
def get_user_prompt():
    return input("Enter your prompt: ")

# Get user prompt
user_prompt = get_user_prompt()

# Apply translation and GPT-3 response generation on each row with the user's prompt
translated_df = df.apply(process_df_row, axis=1, result_type='expand', prompt=user_prompt)

# Concatenate the original DataFrame with the translated DataFrame
final_df = pd.concat([df, translated_df], axis=1)

# # Print the final DataFrame
# print(final_df)

Enter your prompt: This is a spanish medical dataset, I want it translated into english. Every word is in spanish and needs translation. Do not add any additional information, just translate what is written.


In [ ]:
## Awesome!! It works!!!

In [ ]:
##Now lets try to do the same for 50 rows

In [32]:
import pandas as pd

# Read the first row from the CSV file
df = pd.read_csv('To_be_translated.csv',nrows=100)
#df.head()

,ADMISSION_NOTES,TRIAGE_NOTES,BACKGROUND_TRIAGE,OTHER_ALLERGIES
0,DOLOR EN EPIGASTRIO,"Pte con Historia de debilidad general, dolor a...",NaN,Amoxicilina
1,DOLOR EN CUADRANTE INFERIOR IZQUIERDA,Ptes con historia de 1mes de evoluciÃ³n de pre...,Niega patologÃ­as \r\nOperada de apendicitis e...,NaN
2,DIFICULTAD PARA RESPIRAR,Px con dolor en Flanco derecho de 1 dÃ­a de ev...,"Px deambulando, sin signos de bajo gasto",NaN
3,ALERGIA,Pte con hx dolor abdominal generalizado cefale...,Arritmia cardÃ­aca\r\nNiega otra patologia,NaN
4,DOLOR EN FII PRESENTA NAUSEAS Y ESCALOFRIOS,Dolor en cuadrante inferior derecho inicio 5:3...,Ninguna,NaN


In [33]:
df.shape

(100, 4)

In [21]:
## Using the same code as above

import pandas as pd
import requests
import numpy as np

# Function to get response from ChatGPT using a given prompt
def get_chatgpt_response(prompt):
    url = "https://api.openai.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        'Authorization': f'Bearer {api_key}' 
    }
    data = {
        "model": "gpt-3.5-turbo-instruct",  # Adjust the model as needed
        "prompt": prompt,
        "max_tokens": 60
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["text"]
    else:
        print("Failed to get response. Status code:", response.status_code)
        return None

# Function to process each row of the DataFrame
def process_df_row(row, prompt):
    translated_columns = {}
    for col in ['ADMISSION_NOTES', 'TRIAGE_NOTES', 'BACKGROUND_TRIAGE', 'OTHER_ALLERGIES']:
        if pd.notna(row[col]):  # Check if the value is not NaN
            generated_text = get_chatgpt_response(prompt + ' ' + row[col])
            translated_columns[col+'_translated'] = generated_text
        else:
            translated_columns[col+'_translated'] = np.nan
    return translated_columns

# Function to interactively get the prompt from the user
def get_user_prompt():
    return input("Enter your prompt: ")

# Get user prompt
user_prompt = get_user_prompt()

# Apply translation and GPT-3 response generation on each row with the user's prompt
translated_df = df.apply(process_df_row, axis=1, result_type='expand', prompt=user_prompt)

# Concatenate the original DataFrame with the translated DataFrame
final_df = pd.concat([df, translated_df], axis=1)

# # Print the final DataFrame
# print(final_df)

Enter your prompt: This is a spanish medical dataset, I need every word to be translated in english


In [35]:
# Save the DataFrame to an Excel file
final_df.to_excel("final_dataframe.xlsx", index=False)